In [1]:
#!/usr/bin/env python
# encoding: utf-8
#
# Copyright © 2019, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

import swat
import os
from sasctl import Session
from sasctl.tasks import register_model, publish_model
from sasctl.services import microanalytic_score as mas

In [2]:
os.environ["CAS_CLIENT_SSL_CA_LIST"] = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem"

In [3]:
conn = swat.CAS('frasepviya35smp', 8777, username='viyademo01', password='demopw', protocol='https')

In [4]:
s = Session(conn)

In [5]:
from sklearn import linear_model as lm

In [ ]:
tbl = s.CASTable('iris')
if not tbl.tableexists().exists:
    indata = s.upload_file('http://support.sas.com/documentation/onlinedoc/viya/exampledatasets/iris.csv', casout=tbl)

In [28]:
tbl.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species,Index
0,5.1,3.5,1.4,0.2,setosa,1.0
1,4.9,3.0,1.4,0.2,setosa,2.0
2,4.7,3.2,1.3,0.2,setosa,3.0
3,4.6,3.1,1.5,0.2,setosa,4.0
4,5.0,3.6,1.4,0.2,setosa,5.0


In [29]:
s.loadactionset('decisionTree')

tbl.decisiontree.gbtreetrain(target='Species',
                             inputs=['Sepal Length', 'Sepal Width',
                                     'Petal Length', 'Petal Width'],
                             savestate='gradboost_astore')

NOTE: Added action set 'decisionTree'.
NOTE: Wrote 645539 bytes to the savestate file gradboost_astore.


,Descr,Value
0,Number of Trees,150.0
1,Distribution,3.0
2,Learning Rate,0.1
3,Subsampling Rate,0.5
4,Number of Selected Variables (M),4.0
5,Number of Bins,50.0
6,Number of Variables,4.0
7,Max Number of Tree Nodes,15.0
8,Min Number of Tree Nodes,3.0
9,Max Number of Branches,2.0


In [30]:
astore = s.CASTable('gradboost_astore')

In [ ]:
with Session(s):
    
    model = register_model(astore, 'Gradient Boosting', 'Iris')
    
    module = publish_model(model, 'maslocal')
    
    response = mas.execute_module_step(module, 'score',
                                       SepalLength=5.1,
                                       SepalWidth=3.5,
                                       PetalLength=1.4,
                                       PetalWidth=0.2)

In [ ]:
# This is the same as sess.endsession(); sess.close();
s.terminate()